# Decision Tree

In [1]:
import pandas as pd
from plotly import express as px
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv(r"C:\Users\goret\Desktop\APC Cas Kaggle\sigma_cabs.csv")
df  

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,B,1.0,2.42769,A,A,3.90500,0,40.0,46,60,Female,2
1,T0005689461,29.47,B,10.0,2.78245,B,A,3.45000,0,38.0,56,78,Male,2
2,T0005689464,41.58,NaN,10.0,NaN,NaN,E,3.50125,2,NaN,56,77,Male,2
3,T0005689465,61.56,C,10.0,NaN,NaN,A,3.45375,0,NaN,52,74,Male,3
4,T0005689467,54.95,C,10.0,3.03453,B,A,3.40250,4,51.0,49,102,Male,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131657,T0005908509,11.72,D,1.0,2.74229,A,A,3.28500,0,61.0,47,76,Male,3
131658,T0005908510,74.81,C,7.0,2.81059,C,A,0.44500,0,NaN,63,88,Male,2
131659,T0005908512,40.17,C,10.0,2.99565,B,A,3.33625,0,NaN,48,75,Female,2
131660,T0005908513,46.88,B,4.0,3.04744,A,B,4.15750,1,47.0,54,79,Male,2


In [3]:
x = df.iloc[:, 0:12].values
y = df.iloc[:, 12].values

In [4]:
# null values

df["Type_of_Cab"].value_counts()
df["Type_of_Cab"] = df["Type_of_Cab"].fillna("F")

df["Life_Style_Index"].value_counts()
df["Life_Style_Index"].describe()

df["Confidence_Life_Style_Index"].value_counts()

df = df.dropna(subset=["Life_Style_Index"])

sum = 0
count = 0
avg = 0
for  i in df["Customer_Since_Months"].value_counts().iteritems():
    count += i[1]
    sum += i[0]*i[1]
avg = round(sum/count,0)
df["Customer_Since_Months"] = df["Customer_Since_Months"].fillna(avg)

df["Var1"] = df["Var1"].fillna(25)

<ipython-input-4-74b2a62f0cee>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Customer_Since_Months"] = df["Customer_Since_Months"].fillna(avg)
<ipython-input-4-74b2a62f0cee>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Var1"] = df["Var1"].fillna(25)


In [5]:
def one_hot_encoding(column):
    df = pd.get_dummies(column,drop_first=True)
    return df

Type_Of_Cab = one_hot_encoding(df["Type_of_Cab"])
Confidence_Life_Style_Index = one_hot_encoding(df["Confidence_Life_Style_Index"])
Destination_Type = one_hot_encoding(df["Destination_Type"])
Gender = one_hot_encoding(df["Gender"])
Type_Of_Cab = Type_Of_Cab.rename(columns={'B': 'Type_Of_Cab_B','C': 'Type_Of_Cab_C','D': 'Type_Of_Cab_D','E': 'Type_Of_Cab_E','F': 'Type_Of_Cab_F'})
Confidence_Life_Style_Index = Confidence_Life_Style_Index.rename(columns = {"B":"Confidence_Life_Style_Index_B","C":"Confidence_Life_Style_Index_C"})
Destination_Type = Destination_Type.rename(columns = {'B':'Destination_Type_B','C':'Destination_Type_C','D':'Destination_Type_D','E':'Destination_Type_E','F':'Destination_Type_F','G':'Destination_Type_G','H':'Destination_Type_H','I':'Destination_Type_I','J':'Destination_Type_J','K':'Destination_Type_K','L':'Destination_Type_L','M':'Destination_Type_M','N':'Destination_Type_N'})

df_one_hot_encoded = pd.concat([df,Type_Of_Cab,Confidence_Life_Style_Index,Destination_Type,Gender], axis=1)

In [6]:
# normalize

cols_to_drop = ["Trip_ID","Type_of_Cab","Confidence_Life_Style_Index","Destination_Type","Gender"]
df_final = df_one_hot_encoded.drop(cols_to_drop,axis = 1)


cols_to_be_normalized = ['Trip_Distance', 'Customer_Since_Months', 'Life_Style_Index','Customer_Rating', 'Cancellation_Last_1Month', 'Var1', 'Var2', 'Var3']
cols_not_to_be_normalized = ['Type_Of_Cab_B', 'Type_Of_Cab_C', 'Type_Of_Cab_D', 'Type_Of_Cab_E','Type_Of_Cab_F', 
                            'Confidence_Life_Style_Index_B','Confidence_Life_Style_Index_C', 'Destination_Type_B',
                            'Destination_Type_C', 'Destination_Type_D', 'Destination_Type_E',
                            'Destination_Type_F', 'Destination_Type_G', 'Destination_Type_H',
                            'Destination_Type_I', 'Destination_Type_J', 'Destination_Type_K',
                            'Destination_Type_L', 'Destination_Type_M', 'Destination_Type_N',
                            'Male','Surge_Pricing_Type']

normalize = normalize(df_final[cols_to_be_normalized])
normalize = pd.DataFrame(normalize,columns=cols_to_be_normalized)
binarized = df_final[cols_not_to_be_normalized].reset_index()
df_final =  pd.concat([normalize,binarized], axis=1)

df_final

,Trip_Distance,Customer_Since_Months,Life_Style_Index,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,index,Type_Of_Cab_B,...,Destination_Type_G,Destination_Type_H,Destination_Type_I,Destination_Type_J,Destination_Type_K,Destination_Type_L,Destination_Type_M,Destination_Type_N,Male,Surge_Pricing_Type
0,0.078785,0.011637,0.028252,0.045444,0.000000,0.465495,0.535319,0.698242,0,1,...,0,0,0,0,0,0,0,0,0,2
1,0.273009,0.092640,0.025777,0.031961,0.000000,0.352031,0.518783,0.722590,1,1,...,0,0,0,0,0,0,0,0,1,2
2,0.403322,0.073398,0.022273,0.024974,0.029359,0.374330,0.359650,0.748660,4,0,...,0,0,0,0,0,0,0,0,1,2
3,0.234846,0.079019,0.022438,0.023508,0.007902,0.655861,0.395097,0.592646,6,0,...,0,0,0,0,0,0,0,0,1,2
4,0.141184,0.015313,0.021581,0.027429,0.000000,0.788611,0.352195,0.482354,7,1,...,0,0,0,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111464,0.107606,0.009181,0.025178,0.030161,0.000000,0.560065,0.431526,0.697786,131657,0,...,0,0,0,0,0,0,0,0,1,3
111465,0.557730,0.052187,0.020954,0.003318,0.000000,0.186382,0.469683,0.656066,131658,0,...,0,0,0,0,0,0,0,0,1,2
111466,0.396043,0.098592,0.029535,0.032893,0.000000,0.246479,0.473240,0.739437,131659,0,...,0,0,0,0,0,0,0,0,0,2
111467,0.401885,0.034291,0.026125,0.035641,0.008573,0.402913,0.462922,0.677238,131660,1,...,0,0,0,0,0,0,0,0,1,2


In [7]:
df = df.drop(columns = ['Trip_ID'])

In [8]:
df

,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,6.77,B,1.0,2.42769,A,A,3.90500,0,40.0,46,60,Female,2
1,29.47,B,10.0,2.78245,B,A,3.45000,0,38.0,56,78,Male,2
4,54.95,C,10.0,3.03453,B,A,3.40250,4,51.0,49,102,Male,2
6,29.72,E,10.0,2.83958,C,B,2.97500,1,83.0,50,75,Male,2
7,18.44,B,2.0,2.81871,B,A,3.58250,0,103.0,46,63,Male,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131657,11.72,D,1.0,2.74229,A,A,3.28500,0,61.0,47,76,Male,3
131658,74.81,C,7.0,2.81059,C,A,0.44500,0,25.0,63,88,Male,2
131659,40.17,C,10.0,2.99565,B,A,3.33625,0,25.0,48,75,Female,2
131660,46.88,B,4.0,3.04744,A,B,4.15750,1,47.0,54,79,Male,2


### MODEL BUILDING

In [9]:
# funció per calcular el mse

import math

def mean_squeared_error(y1, y2):
    # comprovem que y1 i y2 tenen la mateixa mida
    assert(len(y1) == len(y2))
    mse = 0
    for i in range(len(y1)):
        mse += (y1[i] - y2[i])**2
    return mse / len(y1)

def mse(v1, v2):
    return ((v1 - v2)**2).mean()

In [10]:
X = df_final.drop("Surge_Pricing_Type",axis = 1)
Y = df_final["Surge_Pricing_Type"]

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=8,test_size=0.3)

In [11]:
# Decision tree

model_tree = DecisionTreeClassifier(random_state=0)
model_tree.fit(X_train, Y_train)

DecisionTreeClassifier(random_state=0)

In [12]:
Y_pred1 = model_tree.predict(X_test)

print(confusion_matrix(Y_test, Y_pred1))

[[3561 2062 1378]
 [2228 8529 3701]
 [1248 3632 7102]]


In [13]:
print(accuracy_score(Y_test, Y_pred1))

0.5739062827068568


In [14]:
print(classification_report(Y_test, Y_pred1))

              precision    recall  f1-score   support

           1       0.51      0.51      0.51      7001
           2       0.60      0.59      0.59     14458
           3       0.58      0.59      0.59     11982

    accuracy                           0.57     33441
   macro avg       0.56      0.56      0.56     33441
weighted avg       0.57      0.57      0.57     33441

